In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
main_page = "https://hentaiz.bot/"


In [3]:
regex = 'data-source="https://e.streamqq.com/.*"'

def pull_link_from_text(text:str) -> list[str]:
    matches = re.search(regex, text).group()
    return [matches.removeprefix('data-source="').removesuffix('"')]

def pull_link(link:str) -> list[str]:
    response = requests.get(link)
    text = response.text
    return pull_link_from_text(text)

pull_link("https://hentaiz.bot/hentai/yoasobi-gurashi-1-67b5659807270")

['https://e.streamqq.com/videos/67b4abfe74a2d0cad1041e28/play?event_id=player-wrapper']

In [4]:
def get_page_link(page_num : int)-> str:
    return f"https://hentaiz.bot/?page={page_num}"
test_page = get_page_link(2)
print(test_page)
test_page = requests.get(test_page).text

https://hentaiz.bot/?page=2


In [5]:
'https://hentaiz.bot/hentai/' in test_page

True

In [8]:
content_regex = 'https://hentaiz.bot/hentai/.*"'
image_regex = 'https://hentaiz.bot/img/.*jpg'
def get_detail_page_link(main_page:str) -> list[str]:
    content_matches = re.findall(content_regex,main_page)
    image_matches = re.findall(image_regex,main_page)
    return ([i.removesuffix('"') for i in content_matches],image_matches)
content, images =  get_detail_page_link(test_page)
len(content), len(images)


(48, 48)

In [7]:
title_regex = """
    <head>
        <title>.*</title>
"""
def get_title(text:str)->str:
    return re.search(title_regex,text).group().replace("""<head>
        <title>""","").replace("</title>","").strip()
page = requests.get(content[0]).text
title = get_title(page)
print(title)

Gomu o tsukete to iimashita yo ne... 1 - HentaiZ.bot


In [11]:
#crawl page links
max_page = 111
exception = []
links = []
images = []
for page in range(max_page+1):
    page_text = requests.get(get_page_link(page)).text
    try:
        content, image =  get_detail_page_link(page_text)
        if len(content) == len(image):
            links += content
            images += image
        else:
            exception.append(page)
    except:
        exception.append(page)
    print(page)
print(exception)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
[67]


In [12]:
len(links), len(images)

(5316, 5316)

In [ ]:
video_get_exception = []
videos = []
for page_link in links:
    page_text = requests.get(page_link).text
    try:
        item =  (pull_link_from_text(page_text)[0],get_title(page_text))
        if item not in videos:
            videos.append(item)
        
    except:
        video_get_exception.append(page_link)
video_get_exception,videos

([],
 [('https://e.streamqq.com/videos/68b525c683911575000132f2/play?event_id=player-wrapper',
   'Taishou itsuwari bridal migawari hanayome to gunpuku no mou ai 1 - HentaiZ.bot'),
  ('https://e.streamqq.com/videos/67baa9f1387e33da20073dc2/play?event_id=player-wrapper',
   'Kanojo ga yatsu ni idakareta hi 4 - HentaiZ.bot'),
  ('https://e.streamqq.com/videos/6763e0d306762957d20a8a62/play?event_id=player-wrapper',
   'Bishokuka no rule 1 - HentaiZ.bot'),
  ('https://e.streamqq.com/videos/67be9a38323d63801f0d6686/play?event_id=player-wrapper',
   'Ikusei 1 - HentaiZ.bot'),
  ('https://e.streamqq.com/videos/676c1aae09edd966d1095872/play?event_id=player-wrapper',
   'Onii-san... ohitori desu ka 1 - HentaiZ.bot'),
  ('https://e.streamqq.com/videos/675c5e3db92165cc550ad66c/play?event_id=player-wrapper',
   'Riginetta-san no bouken 2 - HentaiZ.bot'),
  ('https://e.streamqq.com/videos/67614e759e0c76bad00844a2/play?event_id=player-wrapper',
   'Maso mesu soap de aimashou! 1 - HentaiZ.bot'),
  ('

In [24]:
import json
files = open("raw_data.json", 'w')
json.dump({'video_links' : videos, 'video_images' : images},files)
files.close()

In [1]:
import sqlite3
import json
conn = sqlite3.connect('videos_db.db')
cursor = conn.cursor()

file = open("raw_data.json", 'r')

js = json.load(file)

videos= js['video_links']
images = js['video_images']



# Create the videos table if it doesn't exist
# Using AUTOINCREMENT for a unique, auto-increasing primary key
cursor.execute('''
CREATE TABLE IF NOT EXISTS videos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    link TEXT NOT NULL,
    title TEXT NOT NULL,
    image_url TEXT
)
''')

# Clear existing data to prevent duplicates on re-running the script
cursor.execute('DELETE FROM videos')

# Insert the data into the database
# We'll pair each video with an image. If lists are different lengths,
# this will only pair up to the length of the shorter list.
for i in range(min(len(videos), len(images))):
    video_link, video_title = videos[i]
    image_link = images[i*2]
    
    # Using parameterized queries to prevent SQL injection
    cursor.execute(
        "INSERT INTO videos (link, title, image_url) VALUES (?, ?, ?)",
        (video_link, video_title, image_link)
    )

# Commit the changes to the database and close the connection
conn.commit()
conn.close()

print("Database 'videos.db' has been successfully created and populated.")

# --- Optional: Save raw data to JSON ---
# This part matches your original script's intention.
try:
    with open("raw_data.json", 'w') as f:
        json.dump({'video_links': videos, 'video_images': images}, f, indent=4)
    print("Raw data also saved to 'raw_data.json'.")
except Exception as e:
    print(f"Could not write to JSON file: {e}")


Database 'videos.db' has been successfully created and populated.
Raw data also saved to 'raw_data.json'.
